In [3]:
import os
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import lightgbm as lgbm

ModuleNotFoundError: No module named 'lightgbm'

In [103]:
labels = pd.read_csv('../data/processed/labels_matched.csv')
clin = pd.read_csv("../data/raw/clinical.csv", parse_dates=['Case Start', 'Case End'])
clin = clin[clin.Operation.notnull()]

/Users/suzinyou/anaconda2/envs/datathon/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (20,30,32,36,37,38,40,41,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
df = labels[['CaseID']].merge(clin, how='left', on='CaseID')

In [105]:
clin['Age'].isnull().sum()

KeyError: 'Age'

In [13]:
features = ['Age',
 'Height',
 'Weight',
 'BMI',
 'PreopHb',
 'PreopBUN',
 'PreopCr',
 'PreopPT',
 'PreopPTT',
 'PreopGlu']

In [14]:
ff = []
for f in features:
    if f in df.columns:
        ff.append(f)

In [19]:
df = df[['CaseID'] + ff].merge(labels[['CaseID', 'stroke']])

In [20]:
target = df['stroke']
df = df.drop(['stroke'], axis=1)

In [24]:
from sklearn.model_selection import train_test_split

In [147]:
aa = pd.read_csv("/Users/suzinyou/Downloads/clinial_0922.csv")

In [142]:
qq = []
for col in aa.columns:
    any_ = aa[col].apply(is_numeric).any()
    if any_:
        qq.append(col)

In [143]:
aa[qq[10:-2]].head()

,Anesthesia duration,PreopPlt,PreopGPT
CaseID,,,
147,6.58333,263,12
216,1.33333,228,32
217,5,308 Slide 확인함,10
609,2.5,187,10
625,1.08333,307,29


In [138]:
aa = aa.merge(labels, how='inner', on='CaseID')

In [139]:
aa[aa.stroke == 1].shape

(120, 46)

## Which columns have invalid samples?

In [141]:
aa = aa.set_index('CaseID')

In [92]:
features_problematic = qq[10:-2]

In [100]:
pd.set_option('display.max_columns', 999)

In [101]:
overlap = []
od = dict()
for col in features_problematic:
    mask_ = aa[col].apply(is_not_numeric)
    
    intersection = np.intersect1d(aa.index[mask_].values, labels.CaseID.values)
    if len(intersection) > 0:
        display(Markdown("### {}".format(col)))
        display(Markdown("#### Dirty samples"))
        display(aa[mask_])
        display(Markdown("#### Normal samples"))
        display(aa[col][~mask_].sample(6))
        overlap.append(col)
        od[col] = intersection

### Anesthesia duration

#### Dirty samples

,Case Start,Case End,Height,Weight,BMI,AdmDate,DisDate,LOS,PostOpLOS,ICULOS,InHospDeath,ASA,Emergency,Department,Diagnosis,Operation,Surgical position,Anesthesia type,Surgery duration,Anesthesia duration,Hypertension,Diabetes,CVA,Asthma,COPD,LiverDis,KidneyDis,Tb,PreopHb,PreopPlt,PreopAlb,PreopBUN,PreopCr,PreopNa,PreopK,PreopPT,PreopPTT,PreopGlu,PreopGPT,PreopGOT,Sex,Age,source_CaseID,brain_event,stroke
CaseID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28419,2100-01-01 00:00,2100-01-01 12:49,151.7,62.0,26.941423,2099-12-30 16:03:00,2100-01-06 16:03:00,7.0,6.0,1.0,0.0,2.0,N,TS,Lung cancer left,Lobectomy,RLD,General,3.0,#VALUE!,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.7,308,4.1,18.0,0.74,143,4.2,132.0,27.5,117,27,22,F,NaN,14328.0,0.0,0.0


#### Normal samples

CaseID
19480    1.1666666666279
10891            2.08333
32669    4.8333333333721
2649                   3
16696            2.08333
16572                4.5
Name: Anesthesia duration, dtype: object

...

### PreopGPT

#### Dirty samples

,Case Start,Case End,Height,Weight,BMI,AdmDate,DisDate,LOS,PostOpLOS,ICULOS,InHospDeath,ASA,Emergency,Department,Diagnosis,Operation,Surgical position,Anesthesia type,Surgery duration,Anesthesia duration,Hypertension,Diabetes,CVA,Asthma,COPD,LiverDis,KidneyDis,Tb,PreopHb,PreopPlt,PreopAlb,PreopBUN,PreopCr,PreopNa,PreopK,PreopPT,PreopPTT,PreopGlu,PreopGPT,PreopGOT,Sex,Age,source_CaseID,brain_event,stroke
CaseID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11987,2100-01-01 00:00,2100-01-01 15:28,155.0,58.2,24.224766,2099-12-31 11:17:00,2100-09-07 11:17:00,250.0,250.0,251.0,0.0,3.0,Y,NS,Lobar intracerebral hemorrhage,Craniectomy and intracerebral hematoma evacuation,Supine,General,5.500000,6.5,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,12.9,190,2.9,75.0,5.54,129,3.2,80.0,33.8,83,<1,64,F,NaN,NaN,1.0,1.0
11996,2100-01-01 00:00,2100-01-01 10:26,155.0,58.2,24.224766,2099-12-31 01:22:00,2100-09-07 01:22:00,250.0,249.0,251.0,0.0,3.0,Y,NS,Intracerebral hemorrhage,Craniotomy and epidural hemorrhage removal,Supine,General,1.166667,1.5,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,12.9,190,2.9,75.0,5.54,129,3.2,80.0,33.8,83,<1,64,F,NaN,NaN,1.0,1.0
12887,2100-01-01 00:00,2100-01-01 10:27,155.0,57.3,23.850156,2099-12-08 15:57:00,2100-08-15 15:57:00,250.0,227.0,251.0,0.0,3.0,N,NS,Intracerebral hemorrhage,Extracranial ventricular drainage,Supine,General,0.750000,1.41667,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,12.9,190,2.9,75.0,5.54,129,3.2,80.0,33.8,83,<1,64,F,NaN,28993.0,0.0,0.0
13469,2100-01-01 00:00,2100-01-01 13:46,155.0,55.4,23.059313,2099-11-23 09:28:00,2100-07-31 09:28:00,250.0,212.0,251.0,0.0,3.0,Y,GS,Intraventricular hemorrhage,Explorative laparotomy,Lithotomy,General,3.833333,4.81667,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,12.9,190,2.9,75.0,5.54,129,3.2,80.0,33.8,83,<1,64,F,NaN,7956.0,0.0,0.0
16607,2100-01-01 00:00,2100-01-01 09:39,155.0,51.9,21.602497,2099-09-24 13:04:00,2100-06-01 13:04:00,250.0,152.0,251.0,0.0,3.0,N,PS,Pressure sore of coccyx,Debridement of wound,LLD,General,0.583333,0.75,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,12.9,190,2.9,75.0,5.54,129,3.2,80.0,33.8,83,<1,64,F,NaN,NaN,1.0,1.0


#### Normal samples

CaseID
3237     11
27383    12
34391    14
34336    18
29029    18
10151    26
Name: PreopGPT, dtype: object

### Processing rules

1. Don't use anesthesia duration. Highly correlated with surgery duration
2. PreopPlt: take the first present number
3. PreopGPT: '<1' --> '1'

In [94]:
from IPython.display import display, HTML, Markdown

In [59]:
overlap

['Anesthesia duration', 'PreopPlt', 'PreopGPT', 'Age']

In [61]:
od['Age']

array([15018])

In [177]:
def process_age(x):
    if x.endswith('개월'):
        return float(x[:-2]) / 12
    else:
        return float(x)
    
def process_preop_plt(x):
    if isinstance(x, (float, int)):
        return x
    else:
        try:
            return int(x.split()[0])
        except:
            try:
                return int(x.split('p')[0])
            except:
                return int(x.split('c')[0])

def process_preop_gpt(x):
    if x == '<1':
        return 1
    else:
        return x

In [178]:
clin = aa.copy()
clin = clin.merge(labels, how='left', on='CaseID')

In [179]:
clin.loc[:, 'stroke'] = clin.stroke.fillna(0)
clin.loc[:, 'brain_event'] = clin.brain_event.fillna(0)

In [180]:
clin.loc[:, 'Age'] = clin['Age'].apply(process_age)
clin.loc[:, 'PreopPlt'] = clin['PreopPlt'].apply(process_preop_plt)
clin.loc[:, 'PreopGPT'] = clin['PreopGPT'].apply(process_preop_gpt)

In [181]:
clin = clin.drop(['source_CaseID', 'brain_event'], axis=1)

In [182]:
clin.shape

(34980, 44)

In [186]:
clin.isnull().sum()

CaseID                   0
Case Start               0
Case End                 0
Height                   0
Weight                   0
BMI                      0
AdmDate                  0
DisDate                  0
LOS                      0
PostOpLOS                0
ICULOS                   0
InHospDeath              0
ASA                      0
Emergency                0
Department               0
Diagnosis                2
Operation                0
Surgical position      996
Anesthesia type          0
Surgery duration         0
Anesthesia duration      3
Hypertension             0
Diabetes                 0
CVA                      0
Asthma                   0
COPD                     0
LiverDis                 0
KidneyDis                0
Tb                       0
PreopHb                  0
PreopPlt                 0
PreopAlb                 0
PreopBUN                 0
PreopCr                  0
PreopNa                  0
PreopK                   0
PreopPT                  0
P

In [188]:
clin.to_csv("../data/interim/clinical_0935.csv", index=False)

In [189]:
clin.columns

Index(['CaseID', 'Case Start', 'Case End', 'Height', 'Weight', 'BMI',
       'AdmDate', 'DisDate', 'LOS', 'PostOpLOS', 'ICULOS', 'InHospDeath',
       'ASA', 'Emergency', 'Department', 'Diagnosis', 'Operation',
       'Surgical position', 'Anesthesia type', 'Surgery duration',
       'Anesthesia duration', 'Hypertension', 'Diabetes', 'CVA', 'Asthma',
       'COPD', 'LiverDis', 'KidneyDis', 'Tb', 'PreopHb', 'PreopPlt',
       'PreopAlb', 'PreopBUN', 'PreopCr', 'PreopNa', 'PreopK', 'PreopPT',
       'PreopPTT', 'PreopGlu', 'PreopGPT', 'PreopGOT', 'Sex', 'Age', 'stroke'],
      dtype='object')

In [47]:
def is_not_numeric(el):
    try:
        float(str(el))
    except ValueError as e:
        return True
    return False